In [1]:
from torch import float32, max, no_grad, save, device
from torch.backends import mps
from torch.nn import (
    Conv2d,
    CrossEntropyLoss,
    Flatten,
    Linear,
    MaxPool2d,
    Module,
    ReLU,
    Sequential,
    Softmax,
)
from torch.optim import Adam
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import MNIST
from torchvision.transforms.v2 import Compose, Normalize, ToDtype, ToImage

In [2]:
# Define the transformation to apply to the data
transform = Compose(
    [ToImage(), ToDtype(float32, scale=True), Normalize((0.1307,), (0.3081,))]
)

# Load the MNIST dataset
train_data = MNIST("mnist_data", download=True, train=True, transform=transform)
test_data = MNIST("mnist_data", download=True, train=False, transform=transform)

# Split the training set into training and validation sets
train_size = int(0.8 * len(train_data))
val_size = len(train_data) - train_size
trainset, val_data = random_split(train_data, [train_size, val_size])

# Define the data loaders for the training, validation, and testing sets
train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
val_loader = DataLoader(val_data, batch_size=64, shuffle=True)
test_loader = DataLoader(test_data, batch_size=64, shuffle=False)

In [6]:
class CNN(Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.layers = Sequential(
            Conv2d(1, 32, kernel_size=3, padding="same", stride=1), # 28 28 32
            ReLU(inplace=True),
            MaxPool2d(kernel_size=2, stride=2), # 14 14 32
            Conv2d(32, 64, kernel_size=3, padding="same", stride=1), # 14 14 64
            ReLU(inplace=True),
            MaxPool2d(kernel_size=2, stride=2), # 7 7 64
            Flatten(),
            Linear(64 * 7 * 7, 128),
            ReLU(inplace=True),
            Linear(128, 10),
            Softmax(dim=1),
        )

    def forward(self, x):
        x = self.layers(x)
        return x

In [ ]:
device = device("mps" if mps.is_available() else "cpu")
model = CNN().to(device)

In [ ]:
device

In [7]:
# Define hyperparameters
epochs = 10

# Initialize optimizer
optimizer = Adam(model.parameters())

# Define loss function
criterion = CrossEntropyLoss()

Epoch [1/5], Loss: 1.6161
Epoch [2/5], Loss: 1.6441
Epoch [3/5], Loss: 1.4612
Epoch [4/5], Loss: 1.4612
Epoch [5/5], Loss: 1.4918


In [ ]:
# Train the model
model.train()
for epoch in range(epochs):
    for images, labels in train_loader:
        # Forward pass
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        # Backward pass and optimize
        loss.backward()
        optimizer.step()
    print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}")

save(model.state_dict(), "../../../Models/mnist_cnn.pth")

In [8]:
model.eval()

# Track validation accuracy
correct = 0
total = 0

with no_grad():
    for images, labels in val_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Validation Accuracy: {(correct / total):.4f}")

Test Accuracy: 0.9837


In [ ]:
model.eval()

# Track test accuracy
correct = 0
total = 0

with no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Test Accuracy: {(correct / total):.4f}")